In [3]:
import numpy as np
import chemical_potential
import vapour_pressure as vp

### 水分状態を換算するクラス  
換算する前の水分状態と換算後の水分状態を指定し、温度と水分状態を与えることで値を換算するクラス  
$T$：絶対温度[K]  
$R_v$：水蒸気の気体定数  
$RH$：相対湿度[-]  
$P_v$：水蒸気圧[Pa]  
$P_{vs}$：飽和水蒸気圧[Pa]  
$\mu$：水分化学ポテンシャル[J/kg]  

RH（相対湿度） - Pv（水蒸気圧）　関係  
$RH = \frac{P_v}{P_{vs}}$  

RH（相対湿度） - Miu（水分化学ポテンシャル）　関係  
$\mu = R_v T log(RH)$

In [8]:
### 水分状態を換算するクラス ######################
class MoistureConversion():
    Rv = 8316.0/18.016
    def __init__(self, moistureIndexBefore, moistureIndexAfter):
        self.B = moistureIndexBefore
        self.A = moistureIndexAfter
        
    def convert(self, tem, moisture):
#   RH（相対湿度） - Pv（水蒸気圧）　関係
        if self.B == 'RH' and self.A =='Pv':
            pv = moisture* vp.Pvs(tem) 
            return pv
        elif self.B == 'Pv' and self.A =='RH':
            rh = moisture/ vp.Pvs(tem) 
            return rh
#   RH（相対湿度） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'RH' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture) 
            return miu
        elif self.B == 'Miu' and self.A =='RH':
            rh = np.exp(moisture/(self.Rv*tem))
            return rh
#   Pv（水蒸気圧） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'Pv' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture/vp.Pvs(tem)) 
            return miu
        elif self.B == 'Miu' and self.A =='Pv':
            pv = vp.Pvs(tem)*np.exp(moisture/(self.Rv*tem))
            return pv
#   換算が要らない場合
        elif self.B == self.A:
            return moisture
        else : 
            raise ValueError('各物性クラスのmoistureIndexには、「RH」か「Pv」,「Miu」を指定しください。')
################################################

In [16]:
# 使用例
c = MoistureConversion('Miu','RH')
c.convert(293.15, -1500.0)

0.988975971012088

### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  
問題点：  
・ファイルにどの形式の物性値（例えば気相水分伝導率であっても圧力基準なのか密度基準なのか）がわからないこと  
→物性値の形式がわからないため、ファイル中の関数名がわからない（ファイル中に何が入っているか記載しておく？）  
→物性値の種類がどれだけ存在するかわからないため、クラス内に書ききれない（新しい物性値の種類が出た場合は随時追記する？）

注：  
物性値の種類  
・含水率（相対湿度基準・ポテンシャル基準など）  
・液相水分伝導率（含水率基準・ポテンシャル基準など）  
・気相水分伝導率（含水率基準・相対湿度基準・ポテンシャル、温度基準など）

In [26]:
### ある水分状態から物性値を読み取る（換算は無し）
class GetMaterial():

    # 新しく物性モジュールを作成した場合は、ここに追加してください。
    mlist = {
        '合板' : 'Plywood', #物性値ファイルの名前
        'グラスウール' : 'GlassWool',
        'BentheimerSandstone': 'Benstone'
    }
    
    # 問題は取ってきた物性値が何であるか（何に対する液相・気相水分伝導率であるか）
    
    def __init__(self, material, mainMoistureIndex):
        self.material = mlist[material]
        self.B = mainMoistureIndex
        self.A = material.moistureIndex
        self.MC = MoistureConversion(self.B, self.A)

##########################################################
#   熱容量
    def crow(self, tem, moisture):
        return self.material.crow( tem, self.MC.convert(self, tem, moisture) )
#   熱伝導率
    def LAM(self, tem, moisture ):
        return self.material.LAM( tem, self.MC.convert(self, tem, moisture) )
    
##########################################################
#   水分容量（気相の水分状態（ポテンシャル・相対湿度など）から含水率への変換）
    def phi(self, tem, moisture):
        return self.material.phi( tem, self.MC.convert(self, tem, moisture) )
#   含水率から気相の水分状態（ポテンシャル・相対湿度など）への変換）
    def moisture(self, tem, phi):
        return self.material.moisture( tem, phi )
#   液相水分伝導率
    def Pl(self, tem, moisture):
        return self.material.Pl( tem, phi )
#   気相水分伝導率
    def Pg(self, tem, moisture):
        return self.material.Pg( tem, phi )
#   含水率の水分化学ポテンシャル微分
    def DPhi(self, tem,moisture):
        return self.material.DPhi(tem, self.MC.convert(self, tem, moisture))

################################################


### 取得した物性値を流量計算に適した形に換算するクラス（下書き）

In [ ]:
      

####################################################
# 物性値の取得
#   液相水分伝導率（基準：含水率　→　水分化学ポテンシャル）
def LDML(self,DL):
    return DL*row/g
    
#   気相水分伝導率（基準：水蒸気圧　→　水分化学ポテンシャル）
def LDMG(self,Tem,Miu,LDP):
    RH   = chemical_potential.RH_cal(Tem,Miu) 
    DPDM = chemical_potential.DPDM_cal(Tem,RH)
    LDMG = LDP*DPDM
    return LDMG

#   気相水分伝導率（基準：水蒸気圧　→　温度）
def LDTG(self,Tem,Miu,LDP):
    RH   = chemical_potential.RH_cal(Tem,Miu) 
    DPDT = chemical_potential.DPDT_cal(Tem,RH)
    LDTG = LDP*DPDT
    return LDTG
